In [1]:
import os
import random as rand

import django
import flask
import pylibmc
from dash.dependencies import Input, Output, State, MATCH, ALL
from flask_caching import Cache

# to make jupyter-plotly-dash work, had to install jupyter_server_proxy explicitly
# pip install jupyter_server_proxy
# jupyter serverextension enable jupyter_server_proxy
# but honestly it sort of isn't very good anyway and is not presently being used

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mastspec.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from plotter.components import *
from plotter.models import *
from plotter.graph import *
# from plotter.views import *
# from plotter.forms import *
from utils import (   partially_evaluate_from_parameters   )

In [2]:
# initialize the app itself. HTML / react objects must be described in this object
# as dash components.
# callback functions that handle user input must also be described in this object.

app = dash.Dash()

In [3]:
# we are using flask-caching to share state between callbacks,
# because dash refuses to enforce thread safety in python globals and
# so causes problems when globals are set within the execution tree
# of callbacks.
# this is separate from in-browser persistence we will likely add later to 
# protect against, e.g., stray page refreshes.
# this also allows us to cache some expensive things like database lookups.
# (performance has not been profiled here, though.) 
# we could also instead consider memoizing these using standard library
# tools like lru_cache.
# note that we are _not_ presently attempting to make this a multiuser application,
# but a separate cache per user might easily allow that.
# flask-caching requires an explicitly-set 'backend.'
# Our current backend of choice is memcached, which requires a memcached 
# server running on the host. 
# if a memcached install is not desired, don't run this cell, 
# and change 'cache_type' in the cell below.

# initialize a client for caching
# make sure memcached is running with correct parameters
# systemctl edit memcached.service --full on linux, etc.

# current options:
# [Service]
# Environment=OPTIONS="-I 10, -m 1024"
# 1 meg is the default for -I / slab size, which
# was fine for the test set but too small for this set.
# the whole prefetched database is probably around 6M in 
# memory.
# see also memcached-tool 127.0.0.1:11211 settings

# (this can easily be set to start at runtime in a container)
client = pylibmc.Client(["127.0.0.1"], binary=True)

In [4]:
# initialize the cache itself and register it with the app

# change this to 'filesystem' if you don't want to install memcached
cache_type = 'memcached' 

CACHE_CONFIG = {
    'CACHE_TYPE': cache_type,
    'CACHE_DIR': './.cache',
    'CACHE_DEFAULT_TIMEOUT': 0, # keep cached variables for a session of any length
    'SERVERS':client # filesystem backend will just ignore this
}
cache = Cache()
cache.init_app(app.server, config=CACHE_CONFIG)
cache.clear() # this may or may not be desirable in prod, but gives us a clean slate

# cache_set and cache_get are factory functions for setter and getter
# functions for a flask_caching.Cache object.
# in the context of this app initialization, they can be thought
# of as defining a namespace.
cset = cache_set(cache)
cget = cache_get(cache)

In [6]:
spec_model = MSpec

# main_graph is a factory function for a locally-defined dash component.
# dash apps use dash components to automagically generate HTML and 
# react components and handle user I/O at runtime.
# in general our convention is to encapsulate component definitions
# in order to separate them from app definition / initialization.
# we put them in factory functions so that things like html attributes
# can be defined dynamically.
# the exceptions, currently, are the flask route (which, somehow, is
# a component?) and the very top level of app layout.
# this makes it easier for us to move a single component around within 
# the app layout without worrying about changing the component.
# our factory functions for components are stored in plotter.components.
fig = main_graph()

# active queryset is explicitly stored in global cache, as are
# many other app runtime values
cset('queryset', spec_model.objects.all().prefetch_related("observation"))

# this variable is a list of open graph-view tabs;
# these are separate from the 'main' / 'search' tab.
cset('open_graph_viewers', [])

# these are simply lists of inputs that refer to 
# components produced by plotter.components.filter_drop.
# they are defined here for convenience in order to avoid excessive
# repetition in app structure definition and function calls.
# it's possible these should actually be in plotter.components.
x_inputs = [
    Input('filter-1-x', 'value'),
    Input('filter-2-x', 'value'),
    Input('filter-3-x', 'value'),
    Input('axis-option-x', 'value'),
]
y_inputs = [
    Input('filter-1-y', 'value'),
    Input('filter-2-y', 'value'),
    Input('filter-3-y', 'value'),
    Input('axis-option-y', 'value')
]

# client-side url for serving images to the user.
static_image_url = '/images/'

# host-side directory containing those images.
# note this is just ROIs for now
image_directory =  './static_in_pro/our_static/img/roi'

In [7]:
# insert 'settings' / 'global' values for this app into callback functions.
# in Dash, callback functions encapsulate I/O behavior for components and
# are defined separately from components.
# we store our callback functions in plotter.graph.
# typically our convention is that 'global' variables in these functions
# are keyword-only arguments and callback inputs / states are positional
# arguments.

# this is for performance, mostly, and may or may not actually be useful
# at this time
update_spectrum_images = cache.memoize()(update_spectrum_images)
update_spectrum_graph = cache.memoize()(update_spectrum_graph)

settings = {
        'x_inputs': x_inputs,
        'y_inputs': y_inputs,
        'cget': cget,
        'cset': cset,
        # factory functions for plotly figures (which Dash
        # fairly-transparently treats as components).
        # they do not actually fetch or format data;
        # they just do the visual representation.
        'graph_function': main_graph_scatter,
        'spec_graph_function': mspec_graph_line,
        # django model (SQL table + methods) (see plotter.models)
        # containing our spectra.
        # note that insertion of this into functions may end up being
        # a way to generate separate function 'namespaces'
        # in the possible case of, say, wanting to mix mastcam / z data 
        # within a single app instance.
        'spec_model': MSpec,
        'image_directory':  image_directory,
        # scale factor, in viewport units, for spectrum images
        'base_size': 20,
        'static_image_url': static_image_url,
        # file containing saved searches
        'search_file': './saves/saved_searches.csv'
    }
functions_requiring_settings = [
    control_tabs,
    control_search_dropdowns,
    recalculate_graph, 
    update_search_options,
    update_queryset,
    change_calc_input_visibility,
    toggle_search_input_visibility,
    update_spectrum_graph,
    graph_point_to_metadata,
    update_spectrum_images,
    populate_saved_search_drop,
    save_search_tab_state
]
for function in functions_requiring_settings:
    globals()[function.__name__] = partially_evaluate_from_parameters(
        function, settings
    )

In [ ]:
# memoize functions that seem to want caching
# this isn't totally working because the local version isn't mostly what's
# actually being called. doesn't deal with checks in handle_search, etc
# not a big deal right now, will make it work later if necessary
# for function in [
#         recalculate_graph,
#         handle_graph_search,
#         qlist,
#         make_axis,
#         update_search_options,
#         update_queryset
#     ]:
#     function = cache.memoize()(function)

In [8]:
# serve static images using a flask 'route.'
# does defining this function here violate my conventions a little bit? not sure. 
@app.server.route(static_image_url+'<path:path>')
def static_image_link(path):
    static_folder = os.path.join(os.getcwd(), image_directory)
    return flask.send_from_directory(static_folder, path)

In [9]:
# app layout definition 
# the layout property of a dash.Dash object defines how it will 
# lay out its components in the browser at runtime.
# it is nominally equivalent to HTML / DOM tree structure,
# although this gets fuzzy at the pointy end.

# very top level, containing the tabs.
# initialize a 'main' / search tab 
# (using a component factory function from plotter.components).
# all of its children are created within that function
# by calling other component factory functions.

app.layout = html.Div(children = [
    dcc.Tabs(
        children = [search_tab(spec_model)],
        value = 'main_search_tab',
        id = 'tabs'
    ),
])

In [10]:
# callback creation section: register functions from plotter.graph with app i/o

# dash.Dash.callback is a factory function
# that returns an impure function that associates the function
# that is its single argument with the components of that dash.Dash
# object whose ids match the ids of the Output, Input, and State
# objects passed to the callback function.

# syntax for this is:
# app.callback(
    # list of outputs or single output,
    # list of inputs,
    # optional list of states
# )(callback_function)
# when the user interacts with the app, changes in inputs and states
# are passed to callback_function as positional arguments 
# in the order they are given to app.callback.
# elements of whatever callback_function returns are then passed to outputs,
# also in order.
# changes in inputs trigger function evaluation and subsequent output.
# changes in states do not.

# change visibility of x / y axis calculation inputs
# based on arity of calculation function
for axis in ['x', 'y']:
    app.callback(
        [
            Output('filter-1-'+axis, 'style'),
            Output('filter-2-'+axis, 'style'),
            Output('filter-3-'+axis, 'style'),
        ],
        [Input('axis-option-'+axis, 'value')]
    )(change_calc_input_visibility)

# trigger redraw of main graph 
# on new search, axis calculation change, etc
app.callback(
    Output('main-graph', 'figure'),
    # maybe later add an explicit recalc button?
    [
        *x_inputs, 
        *y_inputs, 
        Input({'type':'search-trigger', 'index':ALL}, 'value'),
        Input('main-graph','hoverData')
    ],
    [
        State('main-graph','figure')
    ]
)(recalculate_graph)

# change visibility of search filter inputs 
# based on whether a 'quantitative' or 'qualitative'
# search field is selected
app.callback(
    [
        Output({'type':'term-search', 'index':MATCH},'style'),
        Output({'type':'number-search', 'index':MATCH},'style'),
    ],
    [Input({'type':'field-search', 'index':MATCH},'value')],
    )(toggle_search_input_visibility)

# update displayed search options based on selected search field
app.callback(
    [
        Output({'type':'term-search', 'index':MATCH},'options'),
        Output({'type':'number-range-display','index':MATCH},'children'),
        Output({'type':'number-search', 'index':MATCH},'value'),
    ],
    [
        Input({'type':'field-search', 'index':MATCH},'value'),
        Input({'type':'load-trigger', 'index':0}, 'value')
    ],
    [
        State({'type':'number-search', 'index':MATCH},'value'),
    ]
    )(update_search_options)

# trigger active queryset update on new searches
app.callback(
        Output({'type':'search-trigger', 'index':0},'value'),
        [
            Input({'type':'submit-search', 'index':ALL},'n_clicks'),
            Input({'type':'load-trigger', 'index':0}, 'value')
        ],
        [
            State({'type': 'field-search', 'index': ALL}, 'value'),
            State({'type': 'term-search', 'index': ALL}, 'value'),
            State({'type': 'number-search', 'index': ALL}, 'value'),
            State({'type':'search-trigger', 'index':0},'value')
        ])(update_queryset)

# handle creation and removal of search filters
app.callback(
    [
        Output('search-container', 'children'),
        Output({'type':'submit-search', 'index':1},'n_clicks')
    ],
    [
        Input('add-param', 'n_clicks'), 
        Input({'type':'remove-param', "index":ALL},'n_clicks')
    ],
    [
        State('search-container', 'children'),
        State({'type':'submit-search', 'index':1}, 'n_clicks')
    ]
)(control_search_dropdowns)

# make graph viewer tabs
app.callback(
    [
        Output('tabs', 'children'), 
        Output('tabs', 'value'),
        Output({'type':'load-trigger', 'index':0}, 'value')
    ],
    [
        Input('viewer-open-button','n_clicks'),
        Input({'type':'tab-close-button', "index":ALL},'n_clicks'),
        Input('load-search-load-button', 'n_clicks')
    ],
    [
        State('tabs','children'),
        State('load-search-drop', 'value'),
        State({'type':'load-trigger', 'index':0}, 'value')
    ]
)(control_tabs)

#debug printer
# app.callback(
#     Output('fake-output-for-callback-with-only-side-effects-1', 'children'),
#     [Input('load-search-drop', 'value')]
# )(print_callback)


# point-hover functions.
# right now main and view graph hover functions are basically duplicates, 
# but i'm reserving the possibility that they'll have different behaviors later
app.callback(
    Output({'type':'main-spec-image', 'index':0}, "children"), 
    [Input('main-graph', "hoverData")]
    )(update_spectrum_images)

app.callback(
    Output({'type':'main-spec-print', 'index':0}, "children"), 
    [Input('main-graph', "hoverData")]
    )(graph_point_to_metadata)

app.callback(
    Output({'type':'main-spec-graph','index':0},'figure'),
    [Input('main-graph','hoverData')]
)(update_spectrum_graph)

app.callback(
    Output({'type':'view-spec-image', 'index':MATCH}, "children"), 
    [Input({'type':'view-graph', "index":MATCH},'hoverData')]
    )(update_spectrum_images)

app.callback(
    Output({'type':'view-spec-print', 'index':MATCH}, "children"), 
    [Input({'type':'view-graph', "index":MATCH},'hoverData')]
    )(graph_point_to_metadata)

app.callback(
    Output({'type':'view-spec-graph','index':MATCH},'figure'),
    [Input({'type':'view-graph', "index":MATCH},'hoverData')]
)(update_spectrum_graph)

app.callback(
    Output('fake-output-for-callback-with-only-side-effects-0', 'children'),
    [Input('load-search-save-button', 'n_clicks')]
)(save_search_tab_state)


app.callback(
    Output('load-search-drop', 'options'),
    [Input('load-search-save-button', 'n_clicks')]
    )(populate_saved_search_drop)

# def dummyfunc(data,figure):
#     hovered_point = data["points"][0]["customdata"]
#     figure.update_traces(
#        marker={'color':'red'}
#     )
#     return figure

<function dash.dash.Dash.callback.<locals>.wrap_func.<locals>.add_context(_n_clicks, *, search_file='./saves/saved_searches.csv')>

In [11]:
app.run_server(debug=True, use_reloader = False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [ ]:
app.callback_map

In [ ]:
def robj(model):
    return rand.choice(model.objects.all())